In [1]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import MeshcatVisualizerParams, Role, GeometrySet, CollisionFilterDeclaration
from pydrake.geometry.optimization import CspaceFreePolytope, SeparatingPlaneOrder
from pydrake.multibody.rational import RationalForwardKinematics
from ciris_plant_visualizer import CIrisPlantVisualizer
import numpy as np

from pydrake.all import (HPolyhedron, AngleAxis,
                         VPolytope, Sphere, Ellipsoid, InverseKinematics,
                         Hyperellipsoid, Simulator, Box)
import mcubes

import visualization_utils as viz_utils

import pydrake.symbolic as sym
from pydrake.all import  TriangleSurfaceMesh, Rgba, SurfaceTriangle, Sphere
from scipy.linalg import null_space
import time

In [2]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

In [3]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from pydrake.all import Rgba
from ciris_plant_visualizer import CIrisPlantVisualizer  # Assuming this is set up properly
from visualization_utils import plot_polytope
from pydrake.systems.framework import DiagramBuilder

    

In [4]:
import ipywidgets as widgets
from IPython.display import display

In [5]:
# meshcat = StartMeshcat()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the robot
gripper = parser.AddModels(file_name="my_sdfs/wsg_1dof.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0, 0, 0]))

p_GgraspO = [0, 0, .065]
R_GgraspO = RotationMatrix.MakeXRotation(-np.pi / 2)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_wsg", gripper),
    RigidTransform(R_GgraspO, p_GgraspO))



plant.Finalize()

# Release collision constraints
inspector = scene_graph.model_inspector()

# Penetration allowed between gripper and cap

# Gripper inner collision
# gripper_geometries = GeometrySet()
# for i in range(len(plant.GetBodyIndices(gripper))):
#     body_index = plant.GetBodyIndices(gripper)[i]
#     body_geometries = inspector.GetGeometries(
#         plant.GetBodyFrameIdOrThrow(body_index))
#     gripper_geometries.Add(geometry_ids=body_geometries)
# scene_graph.collision_filter_manager().Apply(
#     CollisionFilterDeclaration().ExcludeWithin(gripper_geometries))

# # Gripper and cap collision
# cap_geometries = GeometrySet()
# for i in range(len(plant.GetBodyIndices(cap))):
#     body_index = plant.GetBodyIndices(cap)[i]
#     body_geometries = inspector.GetGeometries(
#         plant.GetBodyFrameIdOrThrow(body_index))
#     cap_geometries.Add(geometry_ids=body_geometries)
# scene_graph.collision_filter_manager().Apply(
#     CollisionFilterDeclaration().ExcludeWithin(cap_geometries))


# TODO: This will consider the gripper touching the cap as a collision. 
# TODO: Figure out if needs change or adjustment later in grasping space.


# Add visualization
meshcat_params = MeshcatVisualizerParams()
meshcat_params.role = Role.kIllustration

# TCspace
Rat_FK = RationalForwardKinematics(plant)

q_star = np.zeros(plant.num_positions())

# Create the TCspaceFreePolytopes
cspace_free_polytope = CspaceFreePolytope(
    plant, 
    scene_graph,
    SeparatingPlaneOrder.kAffine,
    q_star)

print(plant.num_positions())


visualizer = CIrisPlantVisualizer(
    plant,
    builder,
    scene_graph,
    cspace_free_polytope,
    viz_role=Role.kIllustration
)

plant_context = visualizer.plant_context
diagram = visualizer.task_space_diagram
diagram_context = visualizer.task_space_diagram_context

# Set initial configuration
plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.025
)
# plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
#     plant_context, 0.025
# )
diagram.ForcedPublish(diagram_context)

# visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
# visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(np.pi/2), np.array([0,0,1])))

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001


2


In [6]:


import numpy as np
!pip install plotly
# !pip install ipympl
import plotly.graph_objects as go

from ipywidgets import FloatSlider, VBox, HBox, interact

def visualise_collision_constraint_plotly(factor=1.2, num_points=100):
    # Generate the grid for visualization
    q0 = np.linspace(
        factor * visualizer.q_lower_limits[0],
        factor * visualizer.q_upper_limits[0],
        num_points)
    q1 = np.linspace(
        factor * visualizer.q_lower_limits[1],
        factor * visualizer.q_upper_limits[1],
        num_points)
    X, Y = np.meshgrid(q0, q1)
    Z = np.zeros_like(X)

    # Populate the collision grid
    for i in range(num_points):
        for j in range(num_points):
            # Check for collision: 0 is non-collision, 1 is collision
            Z[i, j] = visualizer.check_collision_q_by_ik(
                np.array([X[i, j], Y[i, j]]))

    # Create the Plotly figure
    fig = go.FigureWidget(
        data=[
            go.Heatmap(
                z=Z,
                x=q0,
                y=q1,
                colorscale=[
                    [0, 'white'],
                    [1, 'red']
                ],
                showscale=False
            ),
            go.Scatter(
                x=[0],  # Initial position
                y=[0],
                mode="markers",
                marker=dict(size=12, color="green"),
                name="Current Position"
            )
        ]
    )

    # Update layout for better readability
    fig.update_layout(
        title="Collision Constraint Visualization",
        xaxis_title="q0",
        yaxis_title="q1",
        autosize=True
    )

    # Initialize sliders for each joint
    sliders = []
    joint_values = np.zeros(2)  # Initialize joint positions (2D example)

    for i in range(len(joint_values)):
        q_low = visualizer.q_lower_limits[i]
        q_high = visualizer.q_upper_limits[i]
        slider = FloatSlider(
            min=q_low, max=q_high, value=0, step=(q_high - q_low) / 100, 
            description=f"q{i}"
        )
        sliders.append(slider)

    # Update slider colors and re-render based on joint state
    def update_plot(*args):
        for i, slider in enumerate(sliders):
            joint_values[i] = slider.value

        # Check if the current joint values are in collision
        in_collision = visualizer.check_collision_q_by_ik(joint_values)
        plant.SetPositions(plant_context, joint_values)
        diagram.ForcedPublish(diagram_context)


        # Update slider color
        for slider in sliders:
            slider.style.handle_color = "orange" if in_collision else "green"

        # Re-render the plot with the new joint values
        fig.data[1].x = [joint_values[0]]
        fig.data[1].y = [joint_values[1]]

        fig.data[1].marker.color = "orange" if in_collision else "green"

    # Attach slider event handlers
    for slider in sliders:
        slider.observe(update_plot, names="value")

    # Display sliders and plot together
    display(VBox(sliders), fig)
    # display(fig)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 14.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.6/305.6 KB 21.7 MB/s eta 0:00:00


In [8]:
%matplotlib

import plotly.io as pio
pio.renderers.default = "notebook"

visualise_collision_constraint_plotly()

Using matplotlib backend: TkAgg
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 KB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.9/774.9 KB 12.9 MB/s eta 0:00:00a 0:00:01


FigureWidget({
    'data': [{'colorscale': [[0, 'white'], [1, 'red']],
              'showscale': False,
              'type': 'heatmap',
              'uid': '00129b56-4133-4e39-b287-ce722fe3339c',
              'x': {'bdata': ('vp8aL90kDsBV4vW+94gNwO0k0U4S7Q' ... 'FOEu0MQFbi9b73iA1Avp8aL90kDkA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('TDeJQWDlsL+HXQFyr7mwv8GDeaL+jb' ... 'zDZ9hVvwDj7MNn2EW/AAAAAAAAAAA='),
                    'dtype': 'f8'},
              'z': {'bdata': ('AAAAAAAA8D8AAAAAAADwPwAAAAAAAP' ... 'AAAADwPwAAAAAAAPA/AAAAAAAA8D8='),
                    'dtype': 'f8',
                    'shape': '100, 100'}},
             {'marker': {'color': 'green', 'size': 12},
              'mode': 'markers',
              'name': 'Current Position',
              'type': 'scatter',
              'uid': '6d5fce66-196a-4def-901b-6f53f0898af1',
              'x': [0],
              'y': [0]}],
    'layout': {'autosize': True,
               'template': '...',
   

In [ ]:
def visualise_collision_constraint_dual_plot(factor=1.2, num_points=100):
    # Generate the grid for q visualization
    q0 = np.linspace(
        factor * visualizer.q_lower_limits[0],
        factor * visualizer.q_upper_limits[0],
        num_points)
    q1 = np.linspace(
        factor * visualizer.q_lower_limits[1],
        factor * visualizer.q_upper_limits[1],
        num_points)
    X_q, Y_q = np.meshgrid(q0, q1)
    Z_q = np.zeros_like(X_q)

    # Populate the collision grid for q
    for i in range(num_points):
        for j in range(num_points):
            Z_q[i, j] = visualizer.check_collision_q_by_ik(
                np.array([X_q[i, j], Y_q[i, j]]))

    # Generate the grid for s visualization
    s0 = np.linspace(
        factor * visualizer.s_lower_limits[0],
        factor * visualizer.s_upper_limits[0],
        num_points)
    s1 = np.linspace(
        factor * visualizer.s_lower_limits[1],
        factor * visualizer.s_upper_limits[1],
        num_points)
    X_s, Y_s = np.meshgrid(s1, s0)  # Invert the axes for s-space
    Z_s = np.zeros_like(X_s)

    # Populate the collision grid for s
    for i in range(num_points):
        for j in range(num_points):
            Z_s[i, j] = visualizer.check_collision_s_by_ik(
                np.array([Y_s[i, j], X_s[i, j]]))  # Note the inversion of s0 and s1

    # Create the q-space plot
    fig_q = go.FigureWidget(
        data=[
            go.Heatmap(
                z=Z_q,
                x=q0,
                y=q1,
                colorscale=[
                    [0, 'white'],
                    [1, 'red']
                ],
                showscale=False
            ),
            go.Scatter(
                x=[0],  # Initial position
                y=[0],
                mode="markers",
                marker=dict(size=12, color="green"),
                name="Current Position"
            )
        ]
    )

    # Create the s-space plot
    fig_s = go.FigureWidget(
        data=[
            go.Heatmap(
                z=Z_s,
                x=s1,  # Horizontal axis for s1
                y=s0,  # Vertical axis for s0
                colorscale=[
                    [0, 'white'],
                    [1, 'red']
                ],
                showscale=False
            ),
            go.Scatter(
                x=[0],  # Initial position (s1)
                y=[0],  # Initial position (s0)
                mode="markers",
                marker=dict(size=12, color="green"),
                name="Current Position"
            )
        ]
    )

    # Update layout for better readability
    for fig in [fig_q, fig_s]:
        fig.update_layout(
            autosize=True,
            margin=dict(l=50, r=50, b=50, t=50)
        )

    plot_width = 500
    fig_q.update_layout(
        title="C-Space Collision Constraint", 
        xaxis_title="q0", 
        yaxis_title="q1",
        width=plot_width
        )
    fig_s.update_layout(
        title="TC-Space Collision Constraint", 
        xaxis_title="s0", 
        yaxis_title="s1",
        width=plot_width
        )  # Inverted axes
    
    # Add placeholder for s0 and s1 values
    fig_s.add_annotation(
        text="s0: 0, s1: 0",  # Placeholder text
        xref="paper", yref="paper",
        x=0.5, y=1.1,  # Position above the plot
        showarrow=False,
        font=dict(size=16)
    )

    # Initialize sliders for each joint
    sliders = []
    joint_values = np.zeros(2)  # Initialize joint positions (2D example)

    for i in range(len(joint_values)):
        q_low = visualizer.q_lower_limits[i]
        q_high = visualizer.q_upper_limits[i]
        slider = FloatSlider(
            min=q_low, max=q_high, value=0, step=(q_high - q_low) / 100, 
            description=f"q{i}"
        )
        sliders.append(slider)

    # Update the plots dynamically based on slider values
    def update_plots(*args):
        for i, slider in enumerate(sliders):
            joint_values[i] = slider.value

        # Compute the s values from the q values using the given function
        s_values = visualizer.rat_forward_kin.ComputeSValue(joint_values, visualizer.q_star)

        # Check collisions
        in_collision_q = visualizer.check_collision_q_by_ik(joint_values)
        in_collision_s = visualizer.check_collision_s_by_ik(s_values)

        # Update q-space marker
        fig_q.data[1].x = [joint_values[0]]
        fig_q.data[1].y = [joint_values[1]]
        fig_q.data[1].marker.color = "orange" if in_collision_q else "green"

        # Update s-space marker
        fig_s.data[1].x = [s_values[1]]  # s1
        fig_s.data[1].y = [s_values[0]]  # s0
        fig_s.data[1].marker.color = "orange" if in_collision_s else "green"

        # Update s-space annotation
        fig_s.layout.annotations[0].text = f"s0: {s_values[1]:.2f}, s1: {s_values[0]:.2f}"

        plant.SetPositions(plant_context, joint_values)
        diagram.ForcedPublish(diagram_context)


        # Update slider colors
        for slider in sliders:
            slider.style.handle_color = "orange" if in_collision_q else "green"

    # Attach slider event handlers
    for slider in sliders:
        slider.observe(update_plots, names="value")

    # Display sliders and both plots
    display(VBox(sliders), HBox([fig_q, fig_s]))

In [ ]:
%matplotlib
visualise_collision_constraint_dual_plot()

In [ ]:
# meshcat = StartMeshcat()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the robot
gripper = parser.AddModels(file_name="my_sdfs/wsg_2dof.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0, 0, 0]))

p_GgraspO = [0, 0, .065]
R_GgraspO = RotationMatrix.MakeXRotation(-np.pi / 2)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_wsg", gripper),
    RigidTransform(R_GgraspO, p_GgraspO))



plant.Finalize()

# Release collision constraints
inspector = scene_graph.model_inspector()

# Penetration allowed between gripper and cap

# Gripper inner collision
# gripper_geometries = GeometrySet()
# for i in range(len(plant.GetBodyIndices(gripper))):
#     body_index = plant.GetBodyIndices(gripper)[i]
#     body_geometries = inspector.GetGeometries(
#         plant.GetBodyFrameIdOrThrow(body_index))
#     gripper_geometries.Add(geometry_ids=body_geometries)
# scene_graph.collision_filter_manager().Apply(
#     CollisionFilterDeclaration().ExcludeWithin(gripper_geometries))

# # Gripper and cap collision
# cap_geometries = GeometrySet()
# for i in range(len(plant.GetBodyIndices(cap))):
#     body_index = plant.GetBodyIndices(cap)[i]
#     body_geometries = inspector.GetGeometries(
#         plant.GetBodyFrameIdOrThrow(body_index))
#     cap_geometries.Add(geometry_ids=body_geometries)
# scene_graph.collision_filter_manager().Apply(
#     CollisionFilterDeclaration().ExcludeWithin(cap_geometries))


# TODO: This will consider the gripper touching the cap as a collision. 
# TODO: Figure out if needs change or adjustment later in grasping space.


# Add visualization
meshcat_params = MeshcatVisualizerParams()
meshcat_params.role = Role.kIllustration

# TCspace
Rat_FK = RationalForwardKinematics(plant)

q_star = np.zeros(plant.num_positions())

# Create the TCspaceFreePolytopes
cspace_free_polytope = CspaceFreePolytope(
    plant, 
    scene_graph,
    SeparatingPlaneOrder.kAffine,
    q_star)

print(plant.num_positions())


visualizer = CIrisPlantVisualizer(
    plant,
    builder,
    scene_graph,
    cspace_free_polytope,
    viz_role=Role.kIllustration
)

plant_context = visualizer.plant_context
diagram = visualizer.task_space_diagram
diagram_context = visualizer.task_space_diagram_context

# Set initial configuration
plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.025
)
# plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
#     plant_context, 0.025
# )
diagram.ForcedPublish(diagram_context)


In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import FloatSlider, VBox, HBox

def visualise_collision_constraint_3D(factor=1, num_points=30):
    # Generate the grid for q visualization (3D)
    q0 = np.linspace(
        factor * visualizer.q_lower_limits[0],
        factor * visualizer.q_upper_limits[0],
        num_points)
    q1 = np.linspace(
        factor * visualizer.q_lower_limits[1],
        factor * visualizer.q_upper_limits[1],
        num_points)
    q2 = np.linspace(
        factor * visualizer.q_lower_limits[2],
        factor * visualizer.q_upper_limits[2],
        num_points)
    
    Q0, Q1, Q2 = np.meshgrid(q0, q1, q2, indexing="ij")
    Z_q = np.zeros_like(Q0)

    # Populate the collision grid for q-space
    for i in range(num_points):
        for j in range(num_points):
            for k in range(num_points):
                Z_q[i, j, k] = visualizer.check_collision_q_by_ik(
                    np.array([Q0[i, j, k], Q1[i, j, k], Q2[i, j, k]]))
                # if q is in the contact manifold (q2 = -0.03), z=2
                if np.isclose(Q2[i, j, k], -0.025, atol=1e-3):
                    Z_q[i, j, k] = 2

    # Generate the grid for s visualization (3D)
    s0 = np.linspace(
        factor * visualizer.s_lower_limits[0],
        factor * visualizer.s_upper_limits[0],
        num_points)
    s1 = np.linspace(
        factor * visualizer.s_lower_limits[1],
        factor * visualizer.s_upper_limits[1],
        num_points)
    s2 = np.linspace(
        factor * visualizer.s_lower_limits[2],
        factor * visualizer.s_upper_limits[2],
        num_points)
    
    S0, S1, S2 = np.meshgrid(s2, s0, s1, indexing="ij")
    Z_s = np.zeros_like(S0)

    # Populate the collision grid for s-space
    for i in range(num_points):
        for j in range(num_points):
            for k in range(num_points):
                Z_s[i, j, k] = visualizer.check_collision_s_by_ik(
                    np.array([S1[i, j, k], S2[i, j, k], S0[i, j, k]]))

    # Create the q-space 3D plot
    fig_q = go.FigureWidget(
        data=[
            go.Volume(
                x=Q0.flatten(),
                y=Q1.flatten(),
                z=Q2.flatten(),
                value=Z_q.flatten(),
                opacity=0.2,  # Adjust transparency for better visibility
                colorscale=[
                    [0, "white"],  # Free space
                    [0.5, "red"],     # Collision space
                    [1, "blue"]     # Contact manifold
                ],
                showscale=False
            ),
            go.Scatter3d(
                x=[0],  # Initial position
                y=[0],
                z=[0],
                mode="markers",
                marker=dict(size=6, color="green"),
                name="Current Position"
            )
        ]
    )

    # Create the s-space 3D plot
    fig_s = go.FigureWidget(
        data=[
            go.Volume(
                x=S0.flatten(),
                y=S1.flatten(),
                z=S2.flatten(),
                value=Z_s.flatten(),
                opacity=0.1,
                colorscale=[
                    [0, "white"],  # Free space
                    [1, "red"]     # Collision space
                ],
                showscale=False
            ),
            go.Scatter3d(
                x=[0],  # Initial position
                y=[0],
                z=[0],
                mode="markers",
                marker=dict(size=6, color="green"),
                name="Current Position"
            )
        ]
    )

    # Set layout properties
    plot_width = 500
    for fig in [fig_q, fig_s]:
        fig.update_layout(
            autosize=True,
            width=plot_width,
            margin=dict(l=50, r=50, b=50, t=50)
        )

    fig_q.update_layout(
        title="C-Space Collision Constraint (3D)", 
        scene=dict(
            xaxis_title="q0", 
            yaxis_title="q1",
            zaxis_title="q2"
        )
    )
    fig_s.update_layout(
        title="TC-Space Collision Constraint (3D)", 
        scene=dict(
            xaxis_title="s0", 
            yaxis_title="s1",
            zaxis_title="s2"
        )
    )
    
    # Add placeholder for s0, s1, and s2 values
    fig_s.add_annotation(
        text="s0: 0, s1: 0, s2: 0",  # Placeholder text
        xref="paper", yref="paper",
        x=0.5, y=1.1,  # Position above the plot
        showarrow=False,
        font=dict(size=14)
    )

    # Initialize sliders for each joint
    sliders = []
    joint_values = np.zeros(3)  # Initialize joint positions (3D)

    for i in range(len(joint_values)):
        q_low = visualizer.q_lower_limits[i]
        q_high = visualizer.q_upper_limits[i]
        slider = FloatSlider(
            min=q_low, max=q_high, value=0, step=(q_high - q_low) / 100, 
            description=f"q{i}"
        )
        sliders.append(slider)

    # Update the plots dynamically based on slider values
    def update_plots(*args):
        for i, slider in enumerate(sliders):
            joint_values[i] = slider.value

        # Compute the s values from the q values using the given function
        s_values = visualizer.rat_forward_kin.ComputeSValue(joint_values, visualizer.q_star)

        # Check collisions
        in_collision_q = visualizer.check_collision_q_by_ik(joint_values)
        in_collision_s = visualizer.check_collision_s_by_ik(s_values)

        # Update q-space marker
        fig_q.data[1].x = [joint_values[0]]
        fig_q.data[1].y = [joint_values[1]]
        fig_q.data[1].z = [joint_values[2]]
        fig_q.data[1].marker.color = "orange" if in_collision_q else "green"

        # Update s-space marker
        fig_s.data[1].x = [s_values[2]]
        fig_s.data[1].y = [s_values[0]]
        fig_s.data[1].z = [s_values[1]]
        fig_s.data[1].marker.color = "orange" if in_collision_s else "green"

        # Update s-space annotation
        fig_s.layout.annotations[0].text = f"s0: {s_values[2]:.2f}, s1: {s_values[0]:.2f}, s2: {s_values[1]:.2f}"

        plant.SetPositions(plant_context, joint_values)
        diagram.ForcedPublish(diagram_context)

        # Update slider colors
        for slider in sliders:
            slider.style.handle_color = "orange" if in_collision_q else "green"

    # Attach slider event handlers
    for slider in sliders:
        slider.observe(update_plots, names="value")

    # Display sliders and both plots
    display(VBox(sliders), HBox([fig_q, fig_s]))



In [ ]:
visualise_collision_constraint_3D()